In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import time
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns
import scipy.stats as stats
from scipy.stats import gamma
from scipy.stats import ortho_group

import cupy as cp
import cupyx.scipy 

from LFHSIC.fhsic_naive import IndpTest_naive_rff
from LFHSIC.lfhsic_g import IndpTest_LFGaussian
from LFHSIC.lfhsic_m import IndpTest_LFMahalanobis

device = torch.device('cuda:0')

In [9]:
# n = 10000, d = 16, D=300
def generate_ISA(n,d,sigma_normal,alpha):
    
    x = np.concatenate((np.random.normal(-1, sigma_normal, n//2), np.random.normal(1, sigma_normal, n//2)))
    y = np.concatenate((np.random.normal(-1, sigma_normal, n//2), np.random.normal(1, sigma_normal, n//2)))
    p = np.random.permutation(n)
    y_p = y[p]

    D = np.zeros([2,n])
    D[0,:] = x
    D[1,:] = y_p

    theta = np.pi/4*alpha
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))

    D_R = R@D
    X_mix = D_R[0,:].reshape(-1,1)
    Y_mix = D_R[1,:].reshape(-1,1)

    X_z = np.random.randn(n,d-1)
    Y_z = np.random.randn(n,d-1)

    X_con = np.concatenate((X_mix,X_z), axis=1)
    Y_con = np.concatenate((Y_mix,Y_z), axis=1)

    m_x = ortho_group.rvs(dim=d)
    m_y = ortho_group.rvs(dim=d)

    X = (m_x@X_con.T).T
    Y = (m_y@Y_con.T).T
    
    return X,Y

# w = 5
def Sinusoid(x, y, w):
    return 1 + np.sin(w*x)*np.sin(w*y)

def Sinusoid_Generator(n,w):
    i = 0
    output = np.zeros([n,2])
    while i < n:
        U = np.random.rand(1)
        V = np.random.rand(2)
        x0 = -np.pi + V[0]*2*np.pi
        x1 = -np.pi + V[1]*2*np.pi
        if U < 1/2 * Sinusoid(x0,x1,w):
            output[i, 0] = x0
            output[i, 1] = x1
            i = i + 1
    return output[:,0], output[:,1]

# n = 3000
# d = 4
def sinedependence(n,d):
    mean = np.zeros(d)
    cov = np.eye(d)
    X = np.random.multivariate_normal(mean, cov, n)
    Z = np.random.randn(n)
    Y = 20*np.sin(4*np.pi*(X[:,0]**2 + X[:,1]**2))+Z 
    return X,Y

# n = 4000
# d = 5
def GSign(n,d):
    mean = np.zeros(d)
    cov = np.eye(d)
    X = np.random.multivariate_normal(mean, cov, n)
    sign_X = np.sign(X)
    Z = np.random.randn(n)
    Y = np.abs(Z)*np.prod(sign_X,1)
    return X,Y

In [14]:
n = 3000
d = 4
D = 100
test_num = 50

result_test_correct = np.zeros([2,test_num])

for it in range(test_num):
    X, Y = sinedependence(n, d)
    Y = Y.reshape(-1,1)
    
    rand_index = np.random.permutation(n)
    X = X[rand_index]
    Y = Y[rand_index]
    X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)

    if len(X_tensor.size())==1:
        X_tensor = X_tensor.reshape(-1,1)
    if len(Y_tensor.size())==1:
        Y_tensor = Y_tensor.reshape(-1,1)

    #test#

    lfhsic_g = IndpTest_LFGaussian(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all1 = lfhsic_g.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[0, it] = float(results_all1['h0_rejected'])

    lfhsic_m = IndpTest_LFMahalanobis(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all2 = lfhsic_m.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[1, it] = float(results_all2['h0_rejected'])

    print(it, np.sum(result_test_correct,1)/(it+1))

0 [0. 0.]
1 [0.  0.5]
2 [0.33333333 0.66666667]
3 [0.25 0.75]
4 [0.2 0.8]
5 [0.33333333 0.66666667]
6 [0.28571429 0.71428571]
7 [0.25 0.75]
8 [0.33333333 0.77777778]
9 [0.4 0.8]
10 [0.36363636 0.72727273]
11 [0.33333333 0.75      ]
12 [0.30769231 0.76923077]
13 [0.28571429 0.78571429]
14 [0.26666667 0.8       ]
15 [0.25   0.8125]
16 [0.23529412 0.82352941]
17 [0.22222222 0.83333333]
18 [0.26315789 0.84210526]
19 [0.3  0.85]
20 [0.28571429 0.85714286]
21 [0.27272727 0.81818182]
22 [0.30434783 0.82608696]
23 [0.33333333 0.83333333]
24 [0.36 0.84]
25 [0.34615385 0.84615385]
26 [0.37037037 0.81481481]
27 [0.35714286 0.78571429]
28 [0.37931034 0.79310345]
29 [0.36666667 0.76666667]
30 [0.35483871 0.77419355]
31 [0.34375 0.75   ]
32 [0.36363636 0.75757576]
33 [0.35294118 0.76470588]
34 [0.37142857 0.77142857]
35 [0.38888889 0.77777778]
36 [0.40540541 0.78378378]
37 [0.39473684 0.76315789]
38 [0.41025641 0.76923077]
39 [0.4   0.775]
40 [0.41463415 0.7804878 ]
41 [0.42857143 0.78571429]
42 [0.

In [15]:
n = 2000
w = 5
D = 100
test_num = 50

result_test_correct = np.zeros([2,test_num])

for it in range(test_num):
    output = Sinusoid_Generator(n,w)
    X = output[0].reshape(-1,1)
    Y = output[1].reshape(-1,1)
    
    rand_index = np.random.permutation(n)
    X = X[rand_index]
    Y = Y[rand_index]
    X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)

    if len(X_tensor.size())==1:
        X_tensor = X_tensor.reshape(-1,1)
    if len(Y_tensor.size())==1:
        Y_tensor = Y_tensor.reshape(-1,1)

    #test#

    lfhsic_g = IndpTest_LFGaussian(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all1 = lfhsic_g.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[0, it] = float(results_all1['h0_rejected'])

    lfhsic_m = IndpTest_LFMahalanobis(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all2 = lfhsic_m.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[1, it] = float(results_all2['h0_rejected'])

    print(it, np.sum(result_test_correct,1)/(it+1))

0 [1. 1.]
1 [1. 1.]
2 [1. 1.]
3 [1. 1.]
4 [1. 1.]
5 [1. 1.]
6 [1. 1.]
7 [1. 1.]
8 [1. 1.]
9 [1. 1.]
10 [1. 1.]
11 [1. 1.]
12 [1. 1.]
13 [1. 1.]
14 [1. 1.]
15 [1. 1.]
16 [1. 1.]
17 [1. 1.]
18 [1. 1.]
19 [1. 1.]
20 [1. 1.]
21 [1. 1.]
22 [1. 1.]
23 [1. 1.]
24 [1. 1.]
25 [1. 1.]
26 [1. 1.]
27 [1. 1.]
28 [0.96551724 1.        ]
29 [0.96666667 1.        ]
30 [0.96774194 1.        ]
31 [0.96875 1.     ]
32 [0.96969697 1.        ]
33 [0.97058824 1.        ]
34 [0.97142857 1.        ]
35 [0.97222222 1.        ]
36 [0.97297297 1.        ]
37 [0.97368421 1.        ]
38 [0.97435897 1.        ]
39 [0.975 1.   ]
40 [0.97560976 1.        ]
41 [0.97619048 1.        ]
42 [0.97674419 1.        ]
43 [0.97727273 1.        ]
44 [0.97777778 1.        ]
45 [0.97826087 1.        ]
46 [0.95744681 1.        ]
47 [0.95833333 1.        ]
48 [0.95918367 1.        ]
49 [0.96 1.  ]


In [16]:
n = 4000
d = 5
D = 100
test_num = 50

result_test_correct = np.zeros([2,test_num])

for it in range(test_num):
    X, Y = GSign(n,d)
    Y = Y.reshape(-1,1)
    
    rand_index = np.random.permutation(n)
    X = X[rand_index]
    Y = Y[rand_index]
    X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)

    if len(X_tensor.size())==1:
        X_tensor = X_tensor.reshape(-1,1)
    if len(Y_tensor.size())==1:
        Y_tensor = Y_tensor.reshape(-1,1)

    #test#

    lfhsic_g = IndpTest_LFGaussian(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all1 = lfhsic_g.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[0, it] = float(results_all1['h0_rejected'])

    lfhsic_m = IndpTest_LFMahalanobis(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all2 = lfhsic_m.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[1, it] = float(results_all2['h0_rejected'])

    print(it, np.sum(result_test_correct,1)/(it+1))

0 [0. 1.]
1 [0.5 1. ]
2 [0.33333333 1.        ]
3 [0.25 1.  ]
4 [0.2 0.8]
5 [0.16666667 0.83333333]
6 [0.28571429 0.85714286]
7 [0.375 0.75 ]
8 [0.44444444 0.77777778]
9 [0.5 0.8]
10 [0.54545455 0.81818182]
11 [0.58333333 0.83333333]
12 [0.61538462 0.84615385]
13 [0.57142857 0.85714286]
14 [0.53333333 0.86666667]
15 [0.5625 0.875 ]
16 [0.58823529 0.88235294]
17 [0.61111111 0.88888889]
18 [0.57894737 0.89473684]
19 [0.6  0.85]
20 [0.61904762 0.85714286]
21 [0.63636364 0.81818182]
22 [0.65217391 0.7826087 ]
23 [0.66666667 0.75      ]
24 [0.68 0.76]
25 [0.69230769 0.76923077]
26 [0.7037037  0.77777778]
27 [0.71428571 0.78571429]
28 [0.68965517 0.75862069]
29 [0.7        0.76666667]
30 [0.70967742 0.77419355]
31 [0.71875 0.78125]
32 [0.6969697  0.78787879]
33 [0.70588235 0.79411765]
34 [0.71428571 0.8       ]
35 [0.72222222 0.80555556]
36 [0.72972973 0.81081081]
37 [0.73684211 0.81578947]
38 [0.71794872 0.82051282]
39 [0.725 0.825]
40 [0.70731707 0.82926829]
41 [0.71428571 0.83333333]
42 [

In [17]:
# ica experiments #
D = 500
test_num = 50
result_all_correct = []

# alpha_set = np.linspace(0.0, 1.0, num=10)
alpha_set = [1.0]

confident_level = 0.05

for alpha in alpha_set:
    result_test_correct = np.zeros([2,test_num])
    
    for it in range(test_num):
        n = 10000
        d = 16
        sigma_normal = 0.1
        
        X, Y = generate_ISA(n,d,sigma_normal,alpha)
        rand_index = np.random.permutation(n)
        X = X[rand_index]
        Y = Y[rand_index]
        
        X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)
        
        #test#

        lfhsic_g = IndpTest_LFGaussian(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
        results_all1 = lfhsic_g.perform_test(rff_num = D, lr = 0.05, if_grid_search = False, debug = -1)
        result_test_correct[0, it] = float(results_all1['h0_rejected'])

        lfhsic_m = IndpTest_LFMahalanobis(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
        results_all2 = lfhsic_m.perform_test(rff_num = D, lr = 0.05, if_grid_search = False, debug = -1)
        result_test_correct[1, it] = float(results_all2['h0_rejected'])

        print(alpha, it, np.sum(result_test_correct,1)/(it+1))
    
    result_all_correct.append(result_test_correct)

1.0 0 [1. 1.]
1.0 1 [1. 1.]
1.0 2 [0.66666667 1.        ]
1.0 3 [0.75 1.  ]
1.0 4 [0.8 1. ]
1.0 5 [0.83333333 1.        ]
1.0 6 [0.85714286 1.        ]
1.0 7 [0.875 1.   ]
1.0 8 [0.88888889 1.        ]
1.0 9 [0.9 1. ]
1.0 10 [0.81818182 1.        ]
1.0 11 [0.83333333 1.        ]
1.0 12 [0.84615385 1.        ]
1.0 13 [0.78571429 1.        ]
1.0 14 [0.73333333 1.        ]
1.0 15 [0.6875 1.    ]
1.0 16 [0.70588235 1.        ]
1.0 17 [0.72222222 1.        ]
1.0 18 [0.73684211 1.        ]
1.0 19 [0.75 1.  ]
1.0 20 [0.71428571 1.        ]
1.0 21 [0.72727273 1.        ]
1.0 22 [0.73913043 1.        ]
1.0 23 [0.75 1.  ]
1.0 24 [0.72 1.  ]
1.0 25 [0.69230769 1.        ]
1.0 26 [0.7037037 1.       ]
1.0 27 [0.67857143 1.        ]
1.0 28 [0.68965517 1.        ]
1.0 29 [0.7 1. ]
1.0 30 [0.70967742 1.        ]
1.0 31 [0.6875 1.    ]
1.0 32 [0.6969697 1.       ]
1.0 33 [0.67647059 1.        ]
1.0 34 [0.65714286 1.        ]
1.0 35 [0.63888889 1.        ]
1.0 36 [0.64864865 1.        ]
1.0 37 [0.63157